In [ ]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

data_folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, "data_folder"))
sys.path.append(data_folder_path) 

from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import concurrent.futures
plt.rcParams.update({'font.size': 12})

np.set_printoptions(precision=5, suppress=True, linewidth=100)


In [10]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# PARAMETERS TO CHANGE

N = 3
M = 2
V = 1
J_V_final = 0.5
T = 3

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# UNCHANGED PARAMETERS

dt = 0.1
J_V_init = 0.1
mu_V_init = 10
mu_V_final = 0
num_control_points = 12

print(f"N: {N}")
print(f"M: {M}")
print(f"V: {V}")
print(f"J_V_final: {J_V_final}")
print(f"T: {T}")
    
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# FIRST PASS

print("\nROUGH PASS STARTING")
T_start = 1
results = create_optimal_piecewise_linear_paths(N, M, T_start, dt, V, J_V_init, J_V_final, mu_V_init, mu_V_final, num_control_points)
    
times_dense, J_V_path, mu_V_path, obj_value, opt_params, t_control_opt, J_control_opt, mu_control_opt = results
    
hamiltonians = [construct_rescaled_hamiltonian(N, M, V, mu_V_ratio=mu_V_path[i], J_V_ratio=J_V_path[i]) for i in range(len(times_dense))]
final_results = simulate_hamiltonian_time_evolution(hamiltonians, times_dense)
energies, time_evolved_wavefunctions, state_probabilities, state_overlaps, true_energies, ground_state_manifold_overlaps = final_results
gso = ground_state_manifold_overlaps[-1]
ed = energies[-1] - true_energies[-1,0]
print("ROUGH PASS COMPLETED")
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# SECOND PASS

print("\nREFINED PASS STARTING")
results = create_optimal_piecewise_linear_paths(N, M, T, dt, V, J_V_init, J_V_final, mu_V_init, mu_V_final, num_control_points, initial_guess = opt_params)
    
times_dense, J_V_path, mu_V_path, obj_value, opt_params, t_control_opt, J_control_opt, mu_control_opt = results
    
hamiltonians = [construct_rescaled_hamiltonian(N, M, V, mu_V_ratio=mu_V_path[i], J_V_ratio=J_V_path[i]) for i in range(len(times_dense))]
final_results = simulate_hamiltonian_time_evolution(hamiltonians, times_dense)
energies, time_evolved_wavefunctions, state_probabilities, state_overlaps, true_energies, ground_state_manifold_overlaps = final_results
gso = ground_state_manifold_overlaps[-1]
ed = energies[-1] - true_energies[-1,0]
print("REFINED PASS COMPLETED\n")
    
print(f"GROUND_STATE_OVERLAP: {gso:.6f}")
print(f"ENERGY_DIFFERENCE: {ed:.6f}")

n_int = num_control_points - 2
J_int_opt = opt_params[:n_int]
mu_int_opt = opt_params[n_int:2*n_int]
t_int_opt = opt_params[2*n_int:3*n_int]

J_control_opt = np.concatenate(([J_V_init], J_int_opt, [J_V_final]))
mu_control_opt = np.concatenate(([mu_V_init], mu_int_opt, [mu_V_final]))
t_control_opt = np.concatenate(([0.0], t_int_opt, [T]))

print("\nOPTIMAL_J_VALUES:\n", np.array(J_control_opt))
print("\nOPTIMAL_MU_VALUES:\n", np.array(mu_control_opt))
print("\nOPTIMAL_TIMES:\n", np.array(t_control_opt))


N: 3
M: 2
V: 1
J_V_final: 0.5
T: 3

ROUGH PASS STARTING
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.9801532997227018
            Iterations: 76
            Function evaluations: 2494
            Gradient evaluations: 76
Success: True
ROUGH PASS COMPLETED

REFINED PASS STARTING
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.7764324895803051
            Iterations: 59
            Function evaluations: 1852
            Gradient evaluations: 59
Success: True
REFINED PASS COMPLETED

GROUND_STATE_OVERLAP: 0.228297
ENERGY_DIFFERENCE: 1.280090

OPTIMAL_J_VALUES:
 [0.1        0.1751314  0.24751946 0.32008869 0.39492063 0.52992305
 0.77518551 0.77517071 0.77506834 0.7748667  0.77392034 0.5       ]

OPTIMAL_MU_VALUES:
 [10.          9.03176417  8.09789635  7.16168785  6.19701169  4.45708256
  1.29489414  1.29389414  1.29289414  1.29189414  1.29089414  0.        ]

OPTIMAL_TIMES:
 [0.         0.01679572 0.

In [4]:
"""N = 3
M = 2
V = 1
dt = 0.1

J_V_init = 0.1
J_V_final = 0.5
mu_V_init = 10
mu_V_final = 0

num_control_points = 12

Ts = np.linspace(0.1,10,25)
ground_state_overlaps = []
energy_differences = []

opt_params = None

for T in Ts:
    opt_params = None
    
    print(f"\nOptimizing for T = {T}")
    
    results = create_optimal_piecewise_linear_paths(
        N, M, T, dt, V,
        J_V_init, J_V_final,
        mu_V_init, mu_V_final,
        num_control_points,
        initial_guess=opt_params
    )
    
    times_dense, J_V_path, mu_V_path, obj_value, opt_params, t_control_opt, J_control_opt, mu_control_opt = results
    
    hamiltonians = [construct_rescaled_hamiltonian(N, M, V, mu_V_ratio=mu_V_path[i], J_V_ratio=J_V_path[i]) for i in range(len(times_dense))]
    final_results = simulate_hamiltonian_time_evolution(hamiltonians, times_dense)
    energies, time_evolved_wavefunctions, state_probabilities, state_overlaps, true_energies, ground_state_manifold_overlaps = final_results
    gso = ground_state_manifold_overlaps[-1]
    ground_state_overlaps.append(gso)
    ed = energies[-1] - true_energies[-1,0]
    energy_differences.append(ed)
    
    print(f"GROUND_STATE_OVERLAP: {gso:.6f}")
    print(f"ENERGY_DIFFERENCE: {ed:.6f}")
    print()
    
print("All finished:")
print(f"Evolution Times: {Ts}")
print(f"Ground State Overlaps = {ground_state_overlaps}")
print(f"Energy Differences = {ground_state_overlaps}")"""

'N = 3\nM = 2\nV = 1\ndt = 0.1\n\nJ_V_init = 0.1\nJ_V_final = 0.5\nmu_V_init = 10\nmu_V_final = 0\n\nnum_control_points = 12\n\nTs = np.linspace(0.1,10,25)\nground_state_overlaps = []\nenergy_differences = []\n\nopt_params = None\n\nfor T in Ts:\n    opt_params = None\n    \n    print(f"\nOptimizing for T = {T}")\n    \n    results = create_optimal_piecewise_linear_paths(\n        N, M, T, dt, V,\n        J_V_init, J_V_final,\n        mu_V_init, mu_V_final,\n        num_control_points,\n        initial_guess=opt_params\n    )\n    \n    times_dense, J_V_path, mu_V_path, obj_value, opt_params, t_control_opt, J_control_opt, mu_control_opt = results\n    \n    hamiltonians = [construct_rescaled_hamiltonian(N, M, V, mu_V_ratio=mu_V_path[i], J_V_ratio=J_V_path[i]) for i in range(len(times_dense))]\n    final_results = simulate_hamiltonian_time_evolution(hamiltonians, times_dense)\n    energies, time_evolved_wavefunctions, state_probabilities, state_overlaps, true_energies, ground_state_mani

In [5]:
"""plt.figure(figsize=(8,5))
plt.plot(Ts, ground_state_overlaps, marker='o', color = "blue")
plt.xlabel('Total Evolution Time $T$')
plt.ylabel('$\langle \psi(T) | P_D | \psi(T) \\rangle$')
plt.title(f'GS Manifold Overlap vs Total Time: $N = {N}$, $M = {M}$, $V = {V}$, $(J/|V|)_f = {J_V_final}$')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
plt.plot(Ts, energy_differences, marker='o', color = "red")
plt.xlabel('Total Evolution Time $T$')
plt.ylabel('$E(T) - E_0$')
plt.title(f'Energy Difference vs Total Time: $N = {N}$, $M = {M}$, $V = {V}$, $(J/|V|)_f = {J_V_final}$')
plt.grid(True)
plt.tight_layout()
plt.show()"""

'plt.figure(figsize=(8,5))\nplt.plot(Ts, ground_state_overlaps, marker=\'o\', color = "blue")\nplt.xlabel(\'Total Evolution Time $T$\')\nplt.ylabel(\'$\\langle \\psi(T) | P_D | \\psi(T) \\rangle$\')\nplt.title(f\'GS Manifold Overlap vs Total Time: $N = {N}$, $M = {M}$, $V = {V}$, $(J/|V|)_f = {J_V_final}$\')\nplt.grid(True)\nplt.tight_layout()\nplt.show()\n\nplt.figure(figsize=(8,5))\nplt.plot(Ts, energy_differences, marker=\'o\', color = "red")\nplt.xlabel(\'Total Evolution Time $T$\')\nplt.ylabel(\'$E(T) - E_0$\')\nplt.title(f\'Energy Difference vs Total Time: $N = {N}$, $M = {M}$, $V = {V}$, $(J/|V|)_f = {J_V_final}$\')\nplt.grid(True)\nplt.tight_layout()\nplt.show()'